In [ ]:
!pip install efficientnet-pytorch
!pip install albumentations --upgrade
!pip install tez 
!pip install densenet-pytorch

In [ ]:
import os
import albumentations
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import time
import torchvision
from torchvision import datasets, models, transforms
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet
from sklearn import metrics, model_selection, preprocessing
import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping
import cv2 
SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_aug = albumentations.Compose([
            albumentations.RandomResizedCrop(300, 300, interpolation=cv2.INTER_CUBIC),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=0.5)], p=1.)
  
        
valid_aug = albumentations.Compose([
            albumentations.RandomResizedCrop(300, 300,interpolation=cv2.INTER_CUBIC),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

In [ ]:
data_dir = "/content/drive/MyDrive/DataSet_ML/dataset/dataset"

In [ ]:
dfx = pd.read_csv('/content/drive/MyDrive/DataSet_ML/dataset/dataset/train.csv')
mapping_breed = dict((v,k) for k,v in enumerate(dfx.breed.unique()))
df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=0, stratify=dfx.breed.values,shuffle=True
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

image_path = "/content/drive/MyDrive/DataSet_ML/dataset/dataset/train/"
train_image_paths = [os.path.join(image_path, x+'.jpg') for x in df_train.image_id.values]
valid_image_paths = [os.path.join(image_path, x+'.jpg') for x in df_valid.image_id.values]
train_targets = [mapping_breed[x] for x in df_train.breed.values]
valid_targets = [mapping_breed[x] for x in df_valid.breed.values]

In [ ]:
from densenet_pytorch import DenseNet
class SnakeModel_Den161(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.densenet = DenseNet.from_pretrained('densenet161')
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2208 ,  num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        x = self.densenet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [ ]:
class SnakeModelB4(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_pretrained("efficientnet-b4")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(1792,  num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [ ]:
class SnakeModelB5(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_pretrained("efficientnet-b5")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2048 ,  num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [ ]:
class SnakeModelB6(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_pretrained("efficientnet-b6")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2304 ,  num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [ ]:
class SnakeModelB7(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_pretrained("efficientnet-b7")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2560 ,  num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [ ]:
train_dataset = ImageDataset(
    image_paths=train_image_paths,
    targets=train_targets,
    resize=(300,300),
    augmentations=train_aug
)

valid_dataset = ImageDataset(
    image_paths=valid_image_paths,
    targets=valid_targets,
    resize=(300,300),
    augmentations=valid_aug,
)

In [ ]:
dense_161 = SnakeModel_Den161(num_classes=dfx.breed.nunique())
modelB4 = SnakeModelB4(num_classes=dfx.breed.nunique())
modelB5 = SnakeModelB5(num_classes=dfx.breed.nunique())
modelB6 = SnakeModelB6(num_classes=dfx.breed.nunique())
modelB7 = SnakeModelB7(num_classes=dfx.breed.nunique())

In [ ]:
es = EarlyStopping(
    monitor="valid_loss", model_path="modelB4.bin", patience=5, mode="min"
)
modelB4.fit(
    train_dataset,
    valid_dataset=valid_dataset, 
    train_bs=16,
    valid_bs=32,
    device="cuda",
    epochs=60,
    callbacks=[es],
    fp16=True
)
# model.save("model.bin")

In [ ]:
es = EarlyStopping(
    monitor="valid_loss", model_path="modelB5.bin", patience=5, mode="min"
)
modelB5.fit(
    train_dataset,
    valid_dataset=valid_dataset, 
    train_bs=16,
    valid_bs=32,
    device="cuda",
    epochs=60,
    callbacks=[es],
    fp16=True
)
# model.save("model.bin")

In [ ]:
es = EarlyStopping(
    monitor="valid_loss", model_path="modelB6.bin", patience=5, mode="min"
)
modelB6.fit(
    train_dataset,
    valid_dataset=valid_dataset, 
    train_bs=16,
    valid_bs=32,
    device="cuda",
    epochs=60,
    callbacks=[es],
    fp16=True
)
# model.save("model.bin")

In [ ]:

es = EarlyStopping(
    monitor="valid_loss", model_path="modelB7.bin", patience=5, mode="min"
)
modelB7.fit(
    train_dataset,
    valid_dataset=valid_dataset, 
    train_bs=8,
    valid_bs=32,
    device="cuda",
    epochs=60,
    callbacks=[es],
    fp16=True,
)
# model.save("model.bin")

In [ ]:

es = EarlyStopping(
    monitor="valid_loss", model_path="dense161.bin", patience=5, mode="min"
)
dense_161.fit(
    train_dataset,
    valid_dataset=valid_dataset, 
    train_bs=16,
    valid_bs=32,
    device="cuda",
    epochs=60,
    callbacks=[es],
    fp16=True
)
# model.save("model.bin")

In [ ]:
test_aug = albumentations.Compose([
    albumentations.RandomResizedCrop(300, 300, interpolation=cv2.INTER_CUBIC),
    albumentations.Transpose(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HueSaturationValue(
        hue_shift_limit=0.2, 
        sat_shift_limit=0.2,
        val_shift_limit=0.2, 
        p=0.5
    ),
    albumentations.RandomBrightnessContrast(
        brightness_limit=(-0.1,0.1), 
        contrast_limit=(-0.1, 0.1), 
        p=0.5
    ),
    albumentations.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225], 
        max_pixel_value=255.0, 
        p=1.0
    )
], p=1.)

In [ ]:
dfx_test = pd.read_csv("/content/drive/MyDrive/DataSet_ML/dataset/dataset/test.csv")
image_path = "/content/drive/MyDrive/DataSet_ML/dataset/dataset/test/"
test_image_paths = [os.path.join(image_path, x+'.jpg') for x in dfx_test.image_id.values]
# fake targets
dfx_test['breed']=[1]*len(dfx_test)
test_targets = dfx_test.breed.values
test_dataset = ImageDataset(
    image_paths=test_image_paths,
    targets=test_targets,
    resize=(300, 300),
    augmentations=test_aug
)

In [ ]:
class SnakeModel_Dense161(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.densenet = DenseNet.from_pretrained('densenet161')
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2208, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.densenet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None

In [ ]:
class SnakeModel_B4(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_name("efficientnet-b4")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(1792, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None

In [ ]:
class SnakeModel_B5(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_name("efficientnet-b5")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2048, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None

In [ ]:
class SnakeModel_B6(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_name("efficientnet-b6")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2304, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None

In [ ]:
class SnakeModel_B7(tez.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.effnet = EfficientNet.from_name("efficientnet-b7")
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(2560, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None

In [ ]:
model_d161 = SnakeModel_Dense161(num_classes=dfx.breed.nunique())
model_d161.load("dense161.bin")

In [ ]:

model_d201 = SnakeModel_Dense201(num_classes=dfx.breed.nunique())
model_d201.load("dense201.bin")

In [ ]:
modelb4 = SnakeModel_B4(num_classes=dfx.breed.nunique())
modelb4.load("modelB4.bin")

In [ ]:
modelb5 = SnakeModel_B5(num_classes=dfx.breed.nunique())
modelb5.load("modelB5.bin")

In [ ]:
modelb6 = SnakeModel_B6(num_classes=dfx.breed.nunique())
modelb6.load("modelB6.bin")

In [ ]:
modelb7 = SnakeModel_B7(num_classes=dfx.breed.nunique())
modelb7.load("modelB7.bin")

In [ ]:
# run inference 5 times
final_preds_d201 = None
for j in range(5):
    preds = model_d201.predict(test_dataset,batch_size=32, n_jobs=-1, device="cuda",sampler=None)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds_d201 is None:
        final_preds_d201 = temp_preds
    else:
        final_preds_d201 += temp_preds
final_preds_d201 /= 5

In [ ]:
# run inference 5 times
final_preds_d161 = None
for j in range(5):
    preds = model_d161.predict(test_dataset,batch_size=32, n_jobs=-1, device="cuda",sampler=None)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds_d161 is None:
        final_preds_d161 = temp_preds
    else:
        final_preds_d161 += temp_preds
final_preds_d161 /= 5

In [ ]:
# run inference 5 times
final_preds_b4 = None
for j in range(5):
    preds = modelb4.predict(test_dataset,batch_size=32, n_jobs=-1, device="cuda",sampler=None)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds_b4 is None:
        final_preds_b4 = temp_preds
    else:
        final_preds_b4 += temp_preds
final_preds_b4 /= 5

In [ ]:
# run inference 5 times
final_preds_b5 = None
for j in range(5):
    preds = modelb5.predict(test_dataset,batch_size=32, n_jobs=-1, device="cuda",sampler=None)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds_b5 is None:
        final_preds_b5 = temp_preds
    else:
        final_preds_b5 += temp_preds
final_preds_b5 /= 5

In [ ]:
# run inference 5 times
final_preds_b6 = None
for j in range(5):
    preds = modelb6.predict(test_dataset,batch_size=32, n_jobs=-1, device="cuda",sampler=None)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds_b6 is None:
        final_preds_b6 = temp_preds
    else:
        final_preds_b6 += temp_preds
final_preds_b6 /= 5

In [ ]:
# run inference 5 times
final_preds_b7 = None
for j in range(5):
    preds = modelb7.predict(test_dataset,batch_size=32, n_jobs=-1, device="cuda",sampler=None)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds_b7 is None:
        final_preds_b7 = temp_preds
    else:
        final_preds_b7 += temp_preds
final_preds_b7 /= 5

In [ ]:
final_preds_ensmb = (final_preds_b6 +  final_preds_b7 + final_preds_d161 + final_preds_d201)/4

final_preds_ensmb = final_preds_ensmb.argmax(axis=1)
dfx_test.breed = final_preds_ensmb
reversed_dictionary = dict(map(reversed, mapping_breed.items()))
dfx_test.breed = dfx_test.breed.apply(lambda x : reversed_dictionary[x])
dfx_test.to_csv("effcnt_ensmb_4.csv",index=False)